In [1]:
import tweepy #https://github.com/tweepy/tweepy
import csv

DEBUG = False

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []    
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    user=api.get_user(screen_name=screen_name)
    
    #retweets=([t.user.screen_name for t in new_tweets[0].retweets()])
    
    #return
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        if DEBUG: print "getting tweets before %s" % (oldest)
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,
                                       count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        if DEBUG: print "...%s tweets downloaded so far" % (len(alltweets))
    
    #transform the tweepy tweets into a 2D array that will populate the csv    
    outtweets = [{'id_str' : tweet.id_str, 
                  'created_at' : tweet.created_at, 
                  'text' : tweet.text.encode("utf-8"),
                  'retweet_count' : tweet.retweet_count,
                  'screen_name' : screen_name}
                 
                 for tweet in alltweets]
    
    
    return outtweets
    
    #write the csv    
#     with open('%s_tweets.csv' % screen_name, 'wb') as f:
#         writer = csv.writer(f)
#         writer.writerow(["id","created_at","text", "retweet_count", "retweet_user_names"])
#         writer.writerows(outtweets)
    
#     pass

tweets = get_all_tweets("djkhaled")

In [2]:
len(tweets)

3197

In [3]:
original_tweets = [t for t in tweets if t['text'][:2] != 'RT']

In [4]:
len(original_tweets)

2662

In [5]:
ids = [tw["id_str"] for tw in original_tweets]

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [7]:
new_ids=[]
for i in ids:
    new_ids.append(i.encode('utf-8'))
new_ids


['836859136516182016',
 '836820822455107584',
 '836818431240507393',
 '836804018076811268',
 '836801386134319104',
 '836769137586135041',
 '836767568639643648',
 '836761383937142784',
 '836735317591875585',
 '836705459314769923',
 '836680612790489088',
 '836467305295069185',
 '836443771726286848',
 '836437035850293249',
 '836407113232629760',
 '836399297470025728',
 '836362292463726592',
 '836345627768311809',
 '836340834760966144',
 '836332821354438656',
 '836331252257079296',
 '836328025398509568',
 '836327779612360705',
 '836322590796963841',
 '836312270175076356',
 '836303515773390850',
 '836303014512111616',
 '836302094906773504',
 '836299200937738241',
 '836288483371200513',
 '836288015773364225',
 '836265301662560256',
 '836252857514930177',
 '836249370425163776',
 '836212223508942848',
 '836200206676946945',
 '836021857149153282',
 '835984586932899841',
 '835963611633823745',
 '835946105829392385',
 '835938344903155713',
 '835927761013342208',
 '835762750752116736',
 '835750556

In [8]:
import time

In [9]:
len(new_ids)

2662

In [23]:
def id_sn_location(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
        
            
        with open('djkhaled2.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [24]:
#id_sn_location(new_ids[300:1001]) 

In [25]:
def id_sn_location1000(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
        
            
        with open('djkhaled1000.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [26]:
#id_sn_location1000(new_ids[1001:])

In [27]:
import pandas as pd

In [28]:
djkhaled=pd.read_csv('djkhaled300.csv')
djkhaled301= pd.read_csv('djkhaled2.csv')
djkhaled1000= pd.read_csv('djkhaled1000.csv')

In [29]:
djkhaled.head()

,id,screen_name,location
0,836859136516182016,GeorgeCuello1,"New York, NY"
1,836859136516182016,kevweaghomi,NaN
2,836859136516182016,kimorosh,"El Paso, TX"
3,836859136516182016,G13MDC,"Griffin, Ga"
4,836859136516182016,obsidianbieber,#1 natalia michalczyk stan


In [30]:
djkhaled301.head()

,id,screen_name,location
0,831008609869787137,cyi_n,Somewhere in Africa.
1,831008609869787137,MaineXXIII,"Dallas, TX"
2,831008609869787137,theboycreator,NaN
3,831008609869787137,DjTreHustle,"Nairobi, Kenya"
4,831008609869787137,mushoe01,Martinsville va


In [31]:
djkhaled1000.head()

,id,screen_name,location
0,813247627718524928,hood_kidd,NaN
1,813247627718524928,Her__BUT,Michigan Detroit
2,813247627718524928,henrygonzalezj1,NYC
3,813247627718524928,flashy_best,"California, USA"
4,813247627718524928,LivingDreems,NaN


In [84]:
djkhaled2=[djkhaled, djkhaled301, djkhaled1000]

In [85]:
thedjkhaled=pd.concat(djkhaled2)
thedjkhaled

,id,screen_name,location
0,836859136516182016,GeorgeCuello1,"New York, NY"
1,836859136516182016,kevweaghomi,NaN
2,836859136516182016,kimorosh,"El Paso, TX"
3,836859136516182016,G13MDC,"Griffin, Ga"
4,836859136516182016,obsidianbieber,#1 natalia michalczyk stan
5,836859136516182016,henrygonzalezj1,NYC
6,836859136516182016,Owenmicheal_,"Swindon, England"
7,836859136516182016,shadesofredblog,New York
8,836859136516182016,aliyu_yoladoka,"Greater Noida, India"
9,836859136516182016,Lilminvj,Riding thru Brooklyn


In [87]:
thedjkhaled.to_pickle('djkhaled.pkl')

In [86]:
len(thedjkhaled['id'].unique())

2661

In [35]:
test=thedjkhaled.groupby(['id'], as_index=False).agg(lambda x: set(x))

In [36]:
test2=test.drop(['location'], axis=1)

In [37]:
test3=test2['screen_name'].str.join(sep='*').str.get_dummies(sep='*')

In [38]:
test4=pd.concat([test2, test3], axis=1)

In [39]:
test4.head()

,id,screen_name,00happyboy,011abubakar,01_chris,035MAJOR,04bxos__,0677792590,0685Goto,06c67cb3f824464,...,zoonthegoon,zrellimehdi,ztUDLG5Yab3XZTB,zul_celeste,zungu_dr,zurinaxxx,zury_agurrie,zustin_jayn,zweediego,zxghklp3
0,778678887991480320,"{thererealboy, biassorac, chatan0065, raydelmo...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,778681155960709120,"{drocktrot, ZaeRetro23_, KdBeatz85, Garcia_Dan...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,778682837864058880,"{kendanw, nickymalik, iYoungNino, krsram, OVOC...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,778684212559818752,"{iluvvcarla, BabybCarson, MisszLlisa, DPeeeZZZ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,778684565334269952,"{thererealboy, drocktrot, CNero_, Ramith_11, V...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
test4.sum(axis=0)

id                 8577875100855199284
00happyboy                           1
011abubakar                          1
01_chris                             1
035MAJOR                             1
04bxos__                             1
0677792590                           1
0685Goto                             1
06c67cb3f824464                      3
07858825817                          1
080Ctouch                            2
100PercentCocky                      1
100stacks_weewe                      1
10101010101j                         1
101Dun                               3
10Omaraziz                           1
10al_                                1
113nonah                             1
117exTakashi                         1
11ShadesOfJ                          1
11_Sharifi                           1
123dice456                           2
13CRUCIFIX                           1
1436Richlane                         3
14kay0                               1
150_atl_shootas          

In [41]:
test4.sum(axis=0).sort_values(ascending=False)

id                 8577875100855199284
henrygonzalezj1                   1633
victorchrisboy1                   1191
reddaakatsuki92                    854
thererealboy                       836
pinkteddy_bella                    643
filafresh                          584
Garcia_Daniel30                    530
joleonbusby                        357
ali_niane15                        344
hood_kidd                          298
Lilminvj                           268
BATMANITSDUMLIT                    222
teddy_bella266                     205
JessicaAubrey6                     192
flashy_best                        191
Lil_CommAS                         181
KingJaxsonn                        178
JaphetLbk                          173
TroubleXKid                        164
VCompteofficiel                    164
jade_becstin                       163
theboycreator                      162
LuvGoddess17                       155
DJZXZQ                             148
Cannabis_Turf            

In [42]:
tweets2 = get_all_tweets("henrygonzalezj1")
tweets3 = get_all_tweets("victorchrisboy1")
tweets4 = get_all_tweets("reddaakatsuki92")

In [43]:
tweets5 = get_all_tweets("thererealboy")
tweets6 = get_all_tweets("pinkteddy_bella")
tweets7 = get_all_tweets("filafresh")

In [44]:
tweets8 = get_all_tweets("Garcia_Daniel30")
tweets9 = get_all_tweets("joleonbusby")
tweets10 = get_all_tweets("ali_niane15")
tweets11 = get_all_tweets("hood_kidd")

In [45]:
print len(tweets2)
print len(tweets3)
print len(tweets4)
print len(tweets5)
print len(tweets6)
print len(tweets7)
print len(tweets8) 
print len(tweets9) 
print len(tweets10)
print len(tweets11)

3246
3230
3218
3054
3250
3204
3248
3200
3208
3199


In [46]:
ogt1 = [t for t in tweets2 if t['text'][:2] != 'RT'] 
ogt2 = [t for t in tweets3 if t['text'][:2] != 'RT']
ogt3 = [t for t in tweets4 if t['text'][:2] != 'RT']
ogt4 = [t for t in tweets5 if t['text'][:2] != 'RT']
ogt5 = [t for t in tweets6 if t['text'][:2] != 'RT']
ogt6 = [t for t in tweets7 if t['text'][:2] != 'RT']
ogt7 = [t for t in tweets8 if t['text'][:2] != 'RT']
ogt8 = [t for t in tweets9 if t['text'][:2] != 'RT']
ogt9 = [t for t in tweets10 if t['text'][:2] != 'RT']
ogt10 = [t for t in tweets11 if t['text'][:2] != 'RT']

In [48]:
print len(tweets2), len(ogt1) 
print len(tweets3), len(ogt2) 
print len(tweets4), len(ogt3) 
print len(tweets5), len(ogt4) 
print len(tweets6), len(ogt5) 
print len(tweets7), len(ogt6) 
print len(tweets8) , len(ogt7) 
print len(tweets9) , len(ogt8) 
print len(tweets10), len(ogt9) 
print len(tweets11), len(ogt10)

3246 336
3230 70
3218 48
3054 43
3250 65
3204 417
3248 28
3200 1
3208 35
3199 10


In [49]:
djkhaled_tweets=pd.DataFrame(original_tweets)
djkhaled_tweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-01 08:42:20,836859136516182016,76,djkhaled,Applauded your self because if you wait for (T...
1,2017-03-01 06:10:06,836820822455107584,32,djkhaled,FANLUV! I'm back in Vegas @marqueelv 3/17 \nTi...
2,2017-03-01 06:00:35,836818431240507393,169,djkhaled,https://t.co/9WGBHT7hQd
3,2017-03-01 05:03:19,836804018076811268,45,djkhaled,Bless up my brother @liltunechi Day 1 real fa...
4,2017-03-01 04:52:52,836801386134319104,22,djkhaled,🙏🏽🔑🙏🏽 https://t.co/DBcypYVdTc


In [52]:
import re

In [55]:
djkhaled_words=[]
for i in djkhaled_tweets['text']:
    djkhaled_words.append(re.sub(r"http\S+","", i))
djkhaled_words

['Applauded your self because if you wait for (THEY\xf0\x9f\x9a\xb7) to applauded you ,you played your self  .. I\xe2\x80\xa6 ',
 "FANLUV! I'm back in Vegas @marqueelv 3/17 \nTickets:  ",
 '',
 'Bless up my brother @liltunechi Day 1  real family ting .. #GRATEFUL THE ALBUM COMING ! \xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd\xf0\x9f\x94\x91\xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd ',
 '\xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd\xf0\x9f\x94\x91\xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd ',
 'Fan luv #DJKHALED FEAT BEYONC\xc3\x89 JAY Z #SHINING ON ITUNES NOW! ',
 '\xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd ',
 '\xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd #GRATEFUL \xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd ',
 'Me and asahdkhaled thanks justinbieber for a amazing video .. top SECRET video and top SECRET\xe2\x80\xa6 ',
 'Congrats !! My partner my friend ! @la_reid ! \xf0\x9f\x94\x91 ',
 'All praise to the most high bless up \xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd',
 '#SHINING Available everywhere ',
 'So you know ! #GRATEFUL ',
 '#DJKHALED #GRATEFUL THE ALBUM COMING

In [56]:
retweeters1=pd.DataFrame(tweets2)
retweeters2=pd.DataFrame(tweets3)
retweeters3=pd.DataFrame(tweets4)
retweeters4=pd.DataFrame(tweets5)
retweeters5=pd.DataFrame(tweets6)
retweeters6=pd.DataFrame(tweets7)
retweeters7=pd.DataFrame(tweets8)
retweeters8=pd.DataFrame(tweets9)
retweeters9=pd.DataFrame(tweets10)
retweeters10=pd.DataFrame(tweets11)

In [57]:
frames=[retweeters1,retweeters2,
    retweeters3,retweeters4,
    retweeters5,retweeters6,
    retweeters7,retweeters8,
    retweeters9,retweeters10]

In [58]:
all_retweets=pd.concat(frames)
all_retweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-03 21:57:35,837784040224747520,39,henrygonzalezj1,RT @RomeoSantosPage: 😎 https://t.co/pjqSlC5mW9
1,2017-03-03 19:46:49,837751131270950912,292,henrygonzalezj1,RT @alexaiono: Watch me on that @nyjets workou...
2,2017-03-03 19:46:36,837751077931986944,347,henrygonzalezj1,RT @enriqueiglesias: Made this @TIDALHiFi play...
3,2017-03-03 19:46:31,837751056926932993,4922,henrygonzalezj1,RT @IHOP: National Pancake Day is back on 3/7!...
4,2017-03-03 19:46:19,837751008059076608,290,henrygonzalezj1,RT @NBA2K: Last night's #NBA2K17 Roster Update...


In [59]:
clean_text=[]
for i in all_retweets['text']:
    clean_text.append(re.sub(r"http\S+","", i))

In [60]:
no_RT=[]
for i in clean_text:
    no_RT.append(re.sub(r"RT", "", i))

In [61]:
retweet_text=pd.DataFrame(no_RT, )

In [62]:
all_retweets['text']=retweet_text
all_retweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-03 21:57:35,837784040224747520,39,henrygonzalezj1,@RomeoSantosPage: 😎
1,2017-03-03 19:46:49,837751131270950912,292,henrygonzalezj1,@alexaiono: Watch me on that @nyjets workout ...
2,2017-03-03 19:46:36,837751077931986944,347,henrygonzalezj1,@enriqueiglesias: Made this @TIDALHiFi playli...
3,2017-03-03 19:46:31,837751056926932993,4922,henrygonzalezj1,@IHOP: National Pancake Day is back on 3/7! E...
4,2017-03-03 19:46:19,837751008059076608,290,henrygonzalezj1,@NBA2K: Last night's #NBA2K17 Roster Update h...


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.corpus import stopwords

In [78]:
stopset = set(stopwords.words('english'))
stopset.update(stopwords.words('spanish'))
stopset.update(['love', 'like', 'new', 'amp', 'get', 'one', 'see', 'si', 'today', 'tonight',
               'go','want','good', 'back', 'thank', 'us', 'day','people','make',
                'much', 'know',])

In [79]:
retweets_str=','.join(no_RT)

In [80]:
vectorizer=TfidfVectorizer(stop_words=stopset,use_idf=True, ngram_range=(1,3))
X=vectorizer.fit_transform([retweets_str])

In [81]:
lsa=TruncatedSVD(n_components=1, n_iter=100, random_state=42)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=1, n_iter=100,
       random_state=42, tol=0.0)

In [88]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:20]
    print("DJ Khaled %d:" % i )
    for term in sortedTerms:
        print(term[0])
    print (" ")

DJ Khaled 0:
wwenetwork
wwe
djkhaled
thequeenmzansi
time
video
right
watch
nba
raw
first
album
live
got
happy
jazz2kool
best
warriors
sdlive
2017
 
